### Adjust Settings

In [6]:
# Turn off ERP/TFR preprocessing if you want to ignore an analysis and save time
do_erp = True
do_tfr = False
# Turn off to prevent plots from saving
save_flag = True
# Turn on for sanity checks (visualizations after each pipeline step that don't get saved)
debug_images = False
# Controls detail level of MNE logs (in order from most to least: debug/info/warning/error/critical)
debug_logs = 'critical'

### Imports

In [7]:
from src.preprocessing import *
from src.plotting import *
import os
import mne
from mne_bids import BIDSPath
import numpy as np
from scipy import stats
from IPython.display import HTML

### Preprocessing Pipeline
For each subject: load their data, filter it, perform ICA and epoching (based on analysis)

In [ ]:
# Set path for BIDS data structure
main_path = BIDSPath(root = 'data', task = 'jacobsen', suffix = 'eeg')
subject_paths = main_path.match()
# !!! 5 subjects break with ICA, we exclude them here to keep the ICA
subjects_to_exclude = [4,8,16,17,18]

# Dictionaries to hold subject data
subject_erp_epochs = {'regular': [], 'random': []}
subject_erp_evokeds = {'regular': [], 'random': []}
subject_tfr_epochs = {'regular': [], 'random': []}
subject_tfr_evokeds = {'regular': [], 'random': []}
subject_tfr_powers = {'regular': [], 'random': []}

# Iterate over subjects
counter = 0
for subject in subject_paths:
        counter += 1
        
        # Load data
        data = load_subject(subject, debug_logs, debug_images)

        # Filter data
        data = filter_data(data, debug_logs, debug_images)

        # Perform independent component analysis
        
        if counter not in subjects_to_exclude:
                data = ica(data, debug_logs, debug_images)

        # Epoch data and derive evokeds for ERP/TFR analysis
        if do_erp:
                # Get epochs for ERP by defining a time window in seconds (X axis) and rejection in microvolts
                epochs_erp = epoch_data(data, time_min = -1, time_max = 1, reject_criteria = 100e-6, debug_logs = debug_logs, debug_images = debug_images)

                subject_erp_epochs['regular'].append(epochs_erp['regular'])
                subject_erp_epochs['random'].append(epochs_erp['random'])
                subject_erp_evokeds['regular'].append(epochs_erp['regular'].average())
                subject_erp_evokeds['random'].append(epochs_erp['random'].average())  
        if do_tfr:
                # Get epochs for TFR by defining a time window in seconds (X axis)
                epochs_tfr = epoch_data(data, time_min = -1, time_max = 3, reject_criteria = False, debug_logs = debug_logs, debug_images = debug_images)
                # Get power spectrum
                powers = perform_tfr(epochs_tfr, debug_logs = debug_logs)

                subject_tfr_epochs['regular'].append(epochs_tfr['regular'])
                subject_tfr_epochs['random'].append(epochs_tfr['random'])
                subject_tfr_evokeds['regular'].append(epochs_tfr['regular'].average())
                subject_tfr_evokeds['random'].append(epochs_tfr['random'].average())
                subject_tfr_powers['regular'].append(powers['regular'])
                subject_tfr_powers['random'].append(powers['random'])

### Analysis: Event-Related Potential (ERP)
Plot and save ERPs of each subject

In [4]:
%%capture
# Remove %%capture to see the cell output i.e. plots

for counter, subject in enumerate(subject_paths):
    
    # Get subject id
    name = f'subject_{counter+1}'
    
    # Get epochs for ERP
    epochs = {'regular': subject_erp_epochs['regular'][counter],
        'random': subject_erp_epochs['random'][counter]}
    
    # Create save directory if it doesn't exist
    os.makedirs(f'plots/{name}', exist_ok = True)

    # Plot ERP
    plot = plot_erp(epochs, subject_id = counter+1)

    # Save plots
    if save_flag:
        save_path = f'plots/{name}/{name}_erp'
        plot.savefig(save_path)

Plot and save Grand Average ERP

In [5]:
%%capture

# Get average
grand_average = {'regular': mne.grand_average(subject_erp_evokeds['regular']), 
            'random': mne.grand_average(subject_erp_evokeds['random'])}
    
# Create save directory if it doesn't exist
os.makedirs(f'plots/subject_average', exist_ok = True)

# Plot average
plot = plot_average_erp(grand_average)

# Save plot
if save_flag:
    save_path = f'plots/subject_average/subject_average_erp' 
    plot.savefig(save_path)

Statistical Significance Testing: Mean Amplitude T-test

In [6]:
# Get trial difference for each subject
diff_waves = []
for counter, _ in enumerate(subject_paths):
    diff_waves.append(mne.combine_evoked([subject_erp_evokeds['regular'][counter],
                                          subject_erp_evokeds['random'][counter]], weights=[1, -1]))
# Channels of interest
picks = ["PO7", "PO8"]

# Get mean amplitude difference
y = np.array([np.mean(e.get_data(picks = picks, tmin = -1, tmax = 1), axis = 1) for e in diff_waves])

# t-Test
t, pval = stats.ttest_1samp(y, 0)
pval = round(pval[0], 4)
print('Difference t =', str(round(t[0], 2)), '/ p =', str(pval))
if pval <= 0.5:
    print('Statistically significant!')
else:
    print('Statistically insignificant!')

Difference t = -4.17 / p = 0.0006
Statistically significant!


### Analysis: Time Frequency Representation (TFR)
Plot and save TFRs for each subject

In [13]:
%%capture

for counter, subject in enumerate(subject_paths):
    
    # Get subject id
    name = f'subject_{counter+1}'
    print(name)
    
    # Get spectra for TFR
    spectra = {'regular': subject_tfr_powers['regular'][counter],
        'random': subject_tfr_powers['random'][counter]}
    
    # Create save directory if it doesn't exist
    os.makedirs(f'plots/{name}', exist_ok = True)

    # Plot TFR
    plot = plot_tfr(spectra, subject_id = counter+1, debug_logs = debug_logs)
    
    # Save plots
    if save_flag:
        save_path = f'plots/{name}/{name}_tfr'
        plot.savefig(save_path)

Plot and save Grand Average TFR + Topographic Map

In [11]:
%%capture

# Get average
grand_average = {'regular': mne.grand_average(subject_tfr_evokeds['regular']), 
            'random': mne.grand_average(subject_tfr_evokeds['random'])}

# Plot average
spectra = perform_tfr(grand_average, debug_logs = debug_logs)
plot = plot_tfr(spectra, subject_id = 'average', debug_logs = debug_logs)

# Plot topomap
plot2 = plot_topomap(grand_average, title = 'Subject average TFR')


# Create save directory if it doesn't exist
os.makedirs(f'plots/subject_average', exist_ok = True)

# Save plots
if save_flag:
    save_path = f'plots/subject_average/subject_average_tfr'
    plot.savefig(save_path)
    save_path2 = f'plots/subject_average/subject_average_topomap'
    plot2.savefig(save_path2)

Animate Topomap

In [ ]:
fig, anim = grand_average['regular'].animate_topomap(ch_type = "eeg", frame_rate = 2, blit = False)
HTML(anim.to_jshtml())